In [17]:
import pandas as pd
import time, datetime

In [3]:
bjt = pd.read_csv("../bdc/hjt.csv")

In [5]:
bjt.head()

,section_id,section_description,travel time,time,lat,long,est_speed,location
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,SECTIONTL0004,M32 Newfoundland Circus IB to Bond St at Glouc...,42.0,07/15/2014 07:09:23 PM +0000,51.4590,-2.585,34.29,"(51.459, -2.585)"
2,SECTIONTL0011,Bond St at Gloucester St EB to M32 Newfoundlan...,51.0,07/15/2014 07:09:23 PM +0000,51.4600,-2.585,28.24,"(51.46, -2.585)"
3,SECTIONTL0012,Old Market St East IB to M32 Newfoundland Circ...,493.0,07/15/2014 07:09:23 PM +0000,51.4580,-2.585,2.92,"(51.458, -2.585)"
4,SECTIONTL0028,Victoria St SB to Old Market St East OB,1335.0,07/15/2014 06:54:23 PM +0000,51.4535,-2.584,3.78,"(51.4535, -2.584)"


### Background ###
* `travel time` is in seconds
* `time` is the beginning of the journey
* `est_speed` is in mph
* `location` can be the same for two different `section_id` as the traffic may be two-directional

### Preprocessing ###
* The first row can be dropped
* `lat` and `long` coluns can be dropped
* Data between *August 205* and *January 2016* should be disregarded -- plenty of errors
* Translate speed to $\frac{m}{s}$
* convert `time` to a timestamp
* Create dictionary `name[section_id] = section_description`

In [6]:
bjt.drop(["lat", "long"], axis=1, inplace=True)

In [8]:
bjt.drop(0, axis=0, inplace=True)

In [30]:
def s2t(s):
    s = s.strip().strip("(").strip(")").split(",")
    s = [float(i.strip()) for i in s]
    s = tuple(s)
    return s
def s2f(s):
    s = float(s)
    # mph to ms
    s *= 0.44704
    return s
def s2i(s):
    return int(s)
def s2d(s):
    if s[-5:] != "+0000":
        print "offset different:", s[-5:]
    return time.mktime(datetime.datetime.strptime(s[:-6], "%m/%d/%Y %I:%M:%S %p").timetuple())

In [31]:
# Convert the timestamp
bjt["time"] = bjt["time"].apply(s2d)

In [36]:
# Drop useless data
# 00:00 01/08/2015 -> 1438387200
# 23:59 31/01/2016 -> 1454284799
def drop_useles(d):
    if d <= 1454284799 and d >= 1438387200:
        return False
    else:
        return True
print bjt.size
bjt = bjt[bjt["time"].map(drop_useles)]
print bjt.size

23582142
18195264


In [41]:
bjt["location"] = bjt["location"].apply(s2t)
bjt["est_speed"] = bjt["est_speed"].apply(s2f)

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [42]:
bjt.head()

,section_id,section_description,travel time,time,est_speed,location
1,SECTIONTL0004,M32 Newfoundland Circus IB to Bond St at Glouc...,42.0,1.405451e+09,15.329002,"(51.459, -2.585)"
2,SECTIONTL0011,Bond St at Gloucester St EB to M32 Newfoundlan...,51.0,1.405451e+09,12.624410,"(51.46, -2.585)"
3,SECTIONTL0012,Old Market St East IB to M32 Newfoundland Circ...,493.0,1.405451e+09,1.305357,"(51.458, -2.585)"
4,SECTIONTL0028,Victoria St SB to Old Market St East OB,1335.0,1.405450e+09,1.689811,"(51.4535, -2.584)"
5,SECTIONTL0030,Victoria St SB to Bath Rd/ Three Lamps OB,207.0,1.405451e+09,10.885424,"(51.4535, -2.584)"


In [43]:
# save the clean data to pickle
bjt.to_pickle("../bdc/hjt_cleaned.pickle")